In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data_path = 'Bike-Sharing-Dataset/hour.csv'

rides = pd.read_csv(data_path)

In [3]:
rides.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [4]:
dummy_fields = ['season', 'weathersit', 'mnth', 'hr', 'weekday']
for each in dummy_fields:
    dummies = pd.get_dummies(rides[each], prefix=each, drop_first=False)
    rides = pd.concat([rides, dummies], axis=1)

fields_to_drop = ['instant', 'dteday', 'season', 'weathersit', 
                  'weekday', 'atemp', 'mnth', 'workingday', 'hr']
data = rides.drop(fields_to_drop, axis=1)
data.head()

,yr,holiday,temp,hum,windspeed,casual,registered,cnt,season_1,season_2,...,hr_21,hr_22,hr_23,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
0,0,0,0.24,0.81,0.0,3,13,16,1,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0.22,0.80,0.0,8,32,40,1,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0.22,0.80,0.0,5,27,32,1,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0.24,0.75,0.0,3,10,13,1,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0.24,0.75,0.0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
quant_features = ['casual', 'registered', 'cnt', 'temp', 'hum', 'windspeed']
# Store scalings in a dictionary so we can convert back later
scaled_features = {}
for each in quant_features:
    mean, std = data[each].mean(), data[each].std()
    scaled_features[each] = [mean, std]
    data.loc[:, each] = (data[each] - mean)/std

In [6]:
# Save the last 21 days 
test_data = data[-21*24:]
data = data[:-21*24]

# Separate the data into features and targets
target_fields = ['cnt', 'casual', 'registered']
features, targets = data.drop(target_fields, axis=1), data[target_fields]
test_features, test_targets = test_data.drop(target_fields, axis=1), test_data[target_fields]

In [7]:
# Hold out the last 60 days of the remaining data as a validation set
train_features, train_targets = features[:-60*24], targets[:-60*24]
val_features, val_targets = features[-60*24:], targets[-60*24:]

In [8]:
train_features.shape

(15435, 56)

In [83]:
sigmoid = lambda x: 1 / (1 - np.exp(-x))

input_nodes = 56
hidden_nodes = 2
output_nodes = 1

inputs_list = train_features.ix[52].values
inputs = np.array(inputs_list, ndmin=2).T

targets_list = train_targets.ix[52]['cnt']
targets = np.array(targets_list, ndmin=2).T


w_input_to_hidden = np.random.normal(scale=hidden_nodes**-0.5, size=(hidden_nodes, input_nodes))
w_hidden_to_output = np.random.normal(scale=output_nodes**-0.5, size=(output_nodes, hidden_nodes))

hidden_inputs = np.dot(w_input_to_hidden, inputs)
hidden_outputs = sigmoid(hidden_inputs)

final_inputs = np.dot(w_hidden_to_output, hidden_outputs)
final_outputs = final_inputs

output_errors = targets - final_outputs
output_grad = output_errors * hidden_outputs.T

hidden_errors =  np.dot(output_errors, w_hidden_to_output).T * hidden_outputs * (1 - hidden_outputs)
hidden_grad = hidden_errors * inputs.T


In [85]:
inputs.shape, targets.shape

((56, 1), (1, 1))

In [86]:
w_input_to_hidden.shape, w_hidden_to_output.shape

((2, 56), (1, 2))

In [87]:
hidden_inputs.shape, hidden_outputs.shape

((2, 1), (2, 1))

In [89]:
final_inputs.shape

(1, 1)

In [90]:
output_errors.shape, output_grad.shape

((1, 1), (1, 2))